In [ ]:
#Loads packages
import numpy as np
import tifffile
import napari
import pandas as pd

from scipy import ndimage as ndi
from skimage.exposure import rescale_intensity
from skimage.morphology import ball, opening,binary_dilation, dilation
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops

In [ ]:
#Function to remove background from imaging data
def remove_backg(img, sig1,sig2):
    img = img.astype('int16')
    img_sig =ndi.gaussian_filter(img, (sig1, sig1,sig1))
    img_bcg = ndi.gaussian_filter(img, (sig2,sig2,sig2))
    cleaned = img_sig - img_bcg
    cleaned[cleaned<0]=0
    return cleaned

In [ ]:
#Function to remove segments smaller than min_size voxels
def erase_small(labels, min_size):
    unique, counts = np.unique(labels, return_counts=True)
    for i in range(len(unique)):
        if counts[i]<min_size:
            labels[labels==unique[i]]=0
    return labels

This notebook is used to perform automated synapse detection guided by immunostaining for synaptic markers. The algorithm uses coCATS (N2V), BASSOON and SHANK2 imaging data as input, as well as manual segmentation of mossy fiber boutons. The output contains pSCR instance segmentation associated with manually segmented boutons.

The test data includes a ROI with a single manually segmented bouton.

#### Load imaging data

In [ ]:
#Loads imaging data
img = tifffile.imread('.\\data\\bassoon_shank2.tif')
bassoon = img[:,0,:,:]
shank2 = img[:,1,:,:]
cats = tifffile.imread('.\\data\\cats_n2v.tif')
segmentation = tifffile.imread('.\\data\\bouton_segmentation.tif')

In [ ]:
#Sets up napari Viewer and displays coCATS imaging data as well as manual MFB segmentation
viewer = napari.Viewer()
viewer.add_image(cats,colormap='gray_r', name='coCATS')
viewer.add_labels(segmentation.astype('int16'), name='MFB segmentation')

#### Pre-process immunostaining data

In [ ]:
#Denoises BASSOON and SHANK2 
sig1 = 1
sig2 = 15
bassoon_cleaned = remove_backg(bassoon, sig1,sig2)
shank2_cleaned = remove_backg(shank2, sig1,sig2)

#Displays denoised data in napari
viewer.add_image(bassoon_cleaned, colormap='magenta', blending = 'additive', name='bassoon cleaned')
viewer.add_image(shank2_cleaned, colormap='cyan', blending = 'additive', name='shank2 cleaned')

In [ ]:
#Thresholds BASSOON and displays the resulting mask in napari
threshhold_bassoon = threshold_otsu(bassoon_cleaned)
mask_bassoon = bassoon_cleaned > threshhold_bassoon
viewer.add_image(mask_bassoon,name='Mask bassoon', colormap='bop purple', blending = 'additive')

#Thresholds SHANK2 and displays the resulting mask in napari
threshhold_shank2 = threshold_otsu(shank2_cleaned)
mask_shank2 = shank2_cleaned > threshhold_shank2
viewer.add_image(mask_shank2,name='Mask shank2', colormap='bop blue', blending = 'additive')

#### Local search

In [ ]:
#Stretches contrast of coCATS data
vmin, vmax = np.quantile(cats, q=(0.01, 0.99))
stretched_cats = rescale_intensity(
    cats, 
    in_range=(vmin, vmax), 
)
#Dilates SHANK2 mask
structure = ball(2)
mask_expanded = binary_dilation(mask_shank2, structure)

#Crops coCATS data according to dilated SHANK2 mask
filt_cats = stretched_cats*mask_expanded
filt_cats[filt_cats<0]=0

#Displays the cropped coCATS data in napari
viewer.add_image(filt_cats,name='Cropped coCATS data', colormap='bop orange', blending = 'additive')

In [ ]:
#Thresholds cropped coCATS data
p95,p100 = np.percentile(filt_cats, (95, 100))
thr1 = (p100-p95)*0.65 + p95 
mask_cats = filt_cats>thr1

#Displays the resulting mask in napari
viewer.add_image(mask_cats,name='Mask coCATS', colormap='bop orange', blending = 'additive')

#### Colocalization analysis

In [ ]:
#Creates instance segmentation of high-intensity coCATS features
labels_cats = label(mask_cats)
labels_cats = erase_small(labels_cats, 8)

In [ ]:
#Identifies pSCRs by colocalization with BASSOON and SHANK2
shank_and_bassoon = mask_bassoon.astype('int')+mask_shank2.astype('int')
cats_props = regionprops(labels_cats, intensity_image = shank_and_bassoon)
cats_triple_colocalized = [x.label for x in cats_props if x.max_intensity > 1]
pSCRs = np.isin(labels_cats,cats_triple_colocalized)
pSCRs = labels_cats*pSCRs

#Displays all pSCRs in napari
viewer.add_labels(pSCRs, name='pSCRs')

#### pSCR - Bouton association

In [ ]:
#Selects pSCRs associated with manually segmented MFBs
expanded_boutons = dilation(segmentation, ball(2))
bouton_props = regionprops(expanded_boutons.astype('int16'),intensity_image = pSCRs)
boutons_stats = pd.DataFrame(columns = ['bouton id', 'number of pSCRs','list of pSCRs'])
for i in range(len(bouton_props)):
    ids_pSCRs_in_bouton = np.unique(bouton_props[i].intensity_image)
    ids_pSCRs_in_bouton = ids_pSCRs_in_bouton[ids_pSCRs_in_bouton!=0]
    data = [[bouton_props[i].label, len(ids_pSCRs_in_bouton),list(ids_pSCRs_in_bouton)]]
    df = pd.DataFrame(data=data, columns = ['bouton id', 'number of pSCRs','list of pSCRs'])
    boutons_stats = boutons_stats.append(df)

pSCRs_in_boutons_ids = boutons_stats['list of pSCRs'].tolist()
pSCRs_in_boutons_ids = [item for sublist in pSCRs_in_boutons_ids for item in sublist]
pSCRs_in_boutons_ids = np.unique(pSCRs_in_boutons_ids)
pSCRs_in_boutons = np.isin(pSCRs,pSCRs_in_boutons_ids)
pSCRs_in_boutons  = pSCRs*pSCRs_in_boutons

#Displays pSCRs associated with manually segmented MFBs in napari
viewer.add_labels(pSCRs_in_boutons, name='pSCRs in boutons')

#### Post-processing for VAST: combine pSCRs and MFBs into one layer

In [ ]:
#Creates a new layer combining MFB and pSCR segments
boutons = np.zeros((segmentation.shape[0], segmentation.shape[1], segmentation.shape[2]), dtype = 'int16')
boutons = segmentation + np.amax(pSCRs_in_boutons)+1
mask = boutons >np.amax(pSCRs_in_boutons)+1
boutons  = boutons *mask 
pSCRs_and_boutons = np.zeros((2, cats.shape[0],cats.shape[1],cats.shape[2]), dtype = 'int16')
pSCRs_and_boutons[0] = boutons
pSCRs_and_boutons[1]= pSCRs_in_boutons
pSCRs_and_boutons = pSCRs_and_boutons.max(0)

#Displays the new layer in napari
viewer.add_labels(pSCRs_and_boutons, name='pSCRs and boutons in one layer')